In [1]:
# %%
from loader.random_circle_loader import RandomCircleDataset
from loader.bptt import BPTTGraphLoader

dataset = RandomCircleDataset('data/random_circle/', subset='val', config='001')
# %%
loader = BPTTGraphLoader(dataset, batch_size=2, shuffle=False, seq_len=5)

data = next(iter(loader))

from models.TactiGraph_lstm import TactiGraph_lstm
model = TactiGraph_lstm()

out = model(data)

/home/hussain/miniconda3/envs/tactile/lib/python3.8/site-packages/torch_geometric/nn/conv/spline_conv.py:133: UserWarning: We do not recommend using the non-optimized CPU version of `SplineConv`. If possible, please move your data to GPU.
  warnings.warn(


torch.Size([5, 2, 512])


In [3]:
out.shape

torch.Size([5, 2, 2])

In [4]:
out

(tensor([[[-0.0436, -0.1060, -0.0655,  ..., -0.2098,  0.0410, -0.0125],
          [ 0.1526,  0.0784, -0.1276,  ..., -0.1336,  0.0417, -0.0231]],
 
         [[ 0.0740, -0.0046, -0.1480,  ..., -0.0726,  0.0370, -0.0200],
          [-0.0684, -0.0466, -0.0611,  ..., -0.0931,  0.2019, -0.0131]],
 
         [[-0.0309, -0.0806, -0.0263,  ..., -0.0981,  0.1427, -0.0045],
          [ 0.1562,  0.0898,  0.1930,  ...,  0.0272,  0.0596, -0.1564]],
 
         [[ 0.1178, -0.0711,  0.3368,  ...,  0.0291, -0.1735, -0.3732],
          [ 0.0942,  0.0417, -0.0094,  ...,  0.0246,  0.4030, -0.0076]],
 
         [[ 0.0499, -0.1277, -0.0117,  ..., -0.0018,  0.5490, -0.0105],
          [ 0.0055, -0.2056, -0.0058,  ..., -0.0650,  0.4243, -0.0611]]],
        grad_fn=<StackBackward0>),
 (tensor([[[ 0.0499, -0.1277, -0.0117,  ..., -0.0018,  0.5490, -0.0105],
           [ 0.0055, -0.2056, -0.0058,  ..., -0.0650,  0.4243, -0.0611]]],
         grad_fn=<StackBackward0>),
  tensor([[[ 0.7160, -0.2534, -0.0871,  ..., -0

In [ ]:
from torch_geometric.data import Data, Batch
from torch_geometric.data.collate import collate

b_0= collate(Batch, data[0])
b_1= collate(Batch, data[1])
collate(Batch, [b_0, b_1])

AttributeError: 'tupleBatch' object has no attribute 'stores_as'